In this exercise we will use Spark to explore the hypothesis stated below:

**The number of farmer's markets in a given zip code can be predicted from the income and taxes paid in a given area.**

We explore the process for discovering whether or not we can accurately predict the number of farmer's markets.  We will use two datasets.

![img](http://training.databricks.com/databricks_guide/USDA_logo.png)

The [**Farmers Markets Directory and Geographic Data**](http://catalog.data.gov/dataset/farmers-markets-geographic-data/resource/cca1cc8a-9670-4a27-a8c7-0c0180459bef) dataset contains information on the longitude and latitude, state, address, name, and zip code of Farmers Markets in the United States. The raw data is published by the Department of Agriculture. 

![img](http://training.databricks.com/databricks_guide/irs-logo.jpg)

The [**SOI Tax Stats - Individual Income Tax Statistics - ZIP Code Data (SOI)**](http://catalog.data.gov/dataset/zip-code-data) study provides detailed tabulations of individual income tax return data at the state and ZIP code level and is provided by the IRS. The data includes items, such as:

- Number of returns, which approximates the number of households
- Number of personal exemptions, which approximates the population
- Adjusted gross income
- Wages and salaries
- Dividends before exclusion
- Interest received

Read in the data

This data is located in in csv files and Apache Spark 2.0 can read the data in directly.

In [3]:
taxes2013 = spark.read\
  .option("header", "true")\
  .csv("dbfs:/databricks-datasets/data.gov/irs_zip_code_data/data-001/2013_soi_zipcode_agi.csv")

In [4]:
markets = spark.read\
  .option("header", "true")\
  .csv("dbfs:/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001/market_data.csv")

Use the `display` command to take a quick look at the dataframes you created.

In [6]:
display(taxes2013)

Now register the DataFrames as Spark SQL tables so that we can use our SQL skills to manipulate the data. The lifetime of this temporary table is tied to the Spark Context that was used to create this DataFrame. When you shutdown the SQLContext associated with a cluster the temporary table disappears as well.

In [8]:
taxes2013.createOrReplaceTempView("taxes2013")
markets.createOrReplaceTempView("markets")

## Running SQL Commands

This is a python notebook.  To use another language in a python notebook we prefix the cell with the language identifier.  To use SQL in your python notebook, prefix the cell with `%sql`

Use SQL to `show tables`.

In [10]:
%sql show tables

Take a quick look at the tables using `SELECT *`.  This is very similar to calling `display` on the DataFrame.

In [12]:
%sql SELECT * FROM taxes2013

Next, cleanup the data using SQL
1. Create a new table called `cleaned_taxes` from the `taxes2013` temp table
1. All the values are currently strings, convert them to approriate data types as needed
  1. zipcode => int
  1. mars1 => int
  1. mars2 => int
  1. numdep => int
  1. A02650 => double
  1. A00300 => double
  1. a01000 => double
  1. a00900 => double 
1. Rename columns for clarity as needed
  1. state => state
  1. zipcode => zipcode
  1. mars1 => single_returns
  1. mars2 => joint_returns
  1. numdep => numdep
  1. A02650 => total_income_amount
  1. A00300 => taxable_interest_amount
  1. a01000 => net_capital_gains
  1. a00900 => biz_net_income
1. Shorten each zip code to 4 digits instead of 5, to group nearby areas together

In [14]:
%sql
DROP TABLE IF EXISTS cleaned_taxes;

CREATE TABLE cleaned_taxes AS
SELECT 
  state, 
  int(zipcode / 10) as zipcode,
  int(mars1) as single_returns,
  int(mars2) as joint_returns,
  int(numdep) as numdep,
  double(A02650) as total_income_amount,
  double(A00300) as taxable_interest_amount,
  double(a01000) as net_capital_gains,
  double(a00900) as biz_net_income
FROM taxes2013

Now that the data is cleaned up, create some nice plots

Explore the average total income per zip code per state. 

Which states have the highest income per zip code?

In [16]:
%sql SELECT state, total_income_amount FROM cleaned_taxes 

New Jersey and California have higher average incomes per zip code.

Next let's explore some specifics of this particular dataset.

Use SQL to `describe` the dataset so that you can see the schema.

In [19]:
%sql describe cleaned_taxes

Let's look at the set of zip codes with the lowest total capital gains and plot the results.

In [21]:
%sql
SELECT zipcode, SUM(net_capital_gains) AS cap_gains
FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY cap_gains ASC
LIMIT 10

Let's look at a combination of capital gains and business net income to see what we find. 

Build a `combo` metric that represents the total capital gains and business net income by zip code.

In [23]:
%sql
SELECT zipcode,
  SUM(biz_net_income) as business_net_income,
  SUM(net_capital_gains) as capital_gains,
  SUM(net_capital_gains) + SUM(biz_net_income) as capital_and_business_income
FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY capital_and_business_income DESC
LIMIT 50

We can also get a peek at what will happen when we execute the query.

Use the `EXPLAIN` keyword in SQL.

In [25]:
%sql
EXPLAIN
  SELECT zipcode,
    SUM(biz_net_income) as net_income,
    SUM(net_capital_gains) as cap_gains,
    SUM(net_capital_gains) + SUM(biz_net_income) as combo
  FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
  GROUP BY zipcode
  ORDER BY combo desc
  limit 50

We can see above that we're fetching the data from `dbfs:/user/hive/warehouse/cleaned_taxes` which is where the data is stored when we registered it as a temporary table. 

Let's `cache` the data

In [27]:
%sql CACHE TABLE cleaned_taxes

When we cache data using SQL, Spark caches *eagerly*--right away.  This differs from `cacheTable` on the `SqlContext` which caches the data only when it is needed.

Let's run the exact same query again. You'll notice that it takes just a fraction of the time because the data is stored in memory.

In [29]:
%sql
SELECT zipcode,
  SUM(biz_net_income) as net_income,
  SUM(net_capital_gains) as cap_gains,
  SUM(net_capital_gains) + SUM(biz_net_income) as combo
FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY combo desc
limit 50

Now `EXPLAIN` the plan

In [31]:
%sql
EXPLAIN
  SELECT zipcode,
    SUM(biz_net_income) as net_income,
    SUM(net_capital_gains) as cap_gains,
    SUM(net_capital_gains) + SUM(biz_net_income) as combo
  FROM cleaned_taxes
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
  GROUP BY zipcode
  ORDER BY combo desc
  limit 50

Instead of going down to the source data it performs a `InMemoryColumnarTableScan`. This means that it has all of the information that it needs in memory.

Now let's look at the Farmer's Market Data. 

Start with a total summation of farmer's markets per state.

In [33]:
%sql SELECT State, COUNT(State) as Sum
      FROM markets 
      GROUP BY State 